# Overdue HW 4 (oops :[ )


## Exercises

You may work in groups for these exercises. They are due at the beginning of next class. You can submit them as a link to a Colab notebook or GitHub CodeSpace.

0. Use the corpora that you assembled last week (Pausanias++):
1. Using programming techniques from the course so far, find other potential collocates for a word of your choice.
2. Calculate the μ and Mutual Information scores for at least 5 of these collocate pairs. How do your results change depending on your definition of a collocation? What might these changes mean? (Write your answers to these questions down.)
3. Calculate the Delta P for these same five pairs. Do any results stand out? Why? What might they tell us about your corpus.


## 0

In [8]:
%pip install -r requirements.txt
%run -m grecy install grc_proiel_sm

INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.1/920.1 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.

Installing grc_proiel_sm.....

Please wait, this could take some minutes.....




In [9]:
from lxml import etree
from MyCapytain.common.constants import Mimetypes
from MyCapytain.resources.texts.local.capitains.cts import CapitainsCtsText
import pandas as pd

with open("tlg0525.tlg001.perseus-grc2.xml") as f:
    text = CapitainsCtsText(urn="urn:cts:greekLit:tlg0525.tlg001.perseus-grc2", resource=f)

urns = []
raw_xmls = []
unannotated_strings = []

for ref in text.getReffs(level=len(text.citation)):
    urn = f"{text.urn}:{ref}"
    node = text.getTextualNode(ref)
    raw_xml = node.export(Mimetypes.XML.TEI)
    tree = node.export(Mimetypes.PYTHON.ETREE)
    s = etree.tostring(tree, encoding="unicode", method="text")

    urns.append(urn)
    raw_xmls.append(raw_xml)
    unannotated_strings.append(s)

d = {
    "urn": pd.Series(urns, dtype="string"),
    "raw_xml": raw_xmls,
    "unannotated_strings": pd.Series(unannotated_strings, dtype="string")
}
pausanias_df = pd.DataFrame(d)
pausanias_df['whitespaced_tokens'] = pausanias_df['unannotated_strings'].str.split()
pausanias_df

,urn,raw_xml,unannotated_strings,whitespaced_tokens
0,urn:cts:greekLit:tlg0525.tlg001.perseus-grc2:1...,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...",τῆς ἠπείρου τῆς Ἑλληνικῆς κατὰ νήσους τὰς Κυκλ...,"[τῆς, ἠπείρου, τῆς, Ἑλληνικῆς, κατὰ, νήσους, τ..."
1,urn:cts:greekLit:tlg0525.tlg001.perseus-grc2:1...,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...","ὁ δὲ Πειραιεὺς δῆμος μὲν ἦν ἐκ παλαιοῦ, πρότερ...","[ὁ, δὲ, Πειραιεὺς, δῆμος, μὲν, ἦν, ἐκ, παλαιοῦ..."
2,urn:cts:greekLit:tlg0525.tlg001.perseus-grc2:1...,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...",θέας δὲ ἄξιον τῶν ἐν Πειραιεῖ μάλιστα Ἀθηνᾶς ἐ...,"[θέας, δὲ, ἄξιον, τῶν, ἐν, Πειραιεῖ, μάλιστα, ..."
3,urn:cts:greekLit:tlg0525.tlg001.perseus-grc2:1...,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...",ἔστι δὲ καὶ ἄλλος Ἀθηναίοις ὁ μὲν ἐπὶ Μουνυχίᾳ...,"[ἔστι, δὲ, καὶ, ἄλλος, Ἀθηναίοις, ὁ, μὲν, ἐπὶ,..."
4,urn:cts:greekLit:tlg0525.tlg001.perseus-grc2:1...,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...",ἀπέχει δὲ σταδίους εἴκοσιν ἄκρα Κωλιάς· ἐς ταύ...,"[ἀπέχει, δὲ, σταδίους, εἴκοσιν, ἄκρα, Κωλιάς·,..."
...,...,...,...,...
3165,urn:cts:greekLit:tlg0525.tlg001.perseus-grc2:1...,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...",οὗτοι μὲν δὴ ὑπεροικοῦσιν Ἀμφίσσης· ἐπὶ θαλάσσ...,"[οὗτοι, μὲν, δὴ, ὑπεροικοῦσιν, Ἀμφίσσης·, ἐπὶ,..."
3166,urn:cts:greekLit:tlg0525.tlg001.perseus-grc2:1...,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...",κληθῆναι δὲ ἀπὸ γυναικὸς ἢ νύμφης τεκμαίρομαι ...,"[κληθῆναι, δὲ, ἀπὸ, γυναικὸς, ἢ, νύμφης, τεκμα..."
3167,urn:cts:greekLit:tlg0525.tlg001.perseus-grc2:1...,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...",τὰ δὲ ἔπη τὰ Ναυπάκτια ὀνομαζόμενα ὑπὸ Ἑλλήνων...,"[τὰ, δὲ, ἔπη, τὰ, Ναυπάκτια, ὀνομαζόμενα, ὑπὸ,..."
3168,urn:cts:greekLit:tlg0525.tlg001.perseus-grc2:1...,"<TEI xmlns=""http://www.tei-c.org/ns/1.0"" xmlns...",ἐνταῦθα ἔστι μὲν ἐπὶ θαλάσσῃ ναὸς Ποσειδῶνος κ...,"[ἐνταῦθα, ἔστι, μὲν, ἐπὶ, θαλάσσῃ, ναὸς, Ποσει..."


In [10]:
# this will take a while

import spacy

nlp = spacy.load("grc_proiel_sm", disable=["ner"])

raw_texts = [t for t in pausanias_df['unannotated_strings']]
annotated_texts = nlp.pipe(raw_texts, batch_size=100)

pausanias_df['nlp_docs'] = list(annotated_texts)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## 1
Using programming techniques from the course so far, find other potential collocates for a word of your choice. 

In [7]:
test_token = pausanias_df['nlp_docs'][0][1]

# we use a list comprehension to evaluate the generator at `test_token.children`
f"token: '{test_token}, {test_token.lemma_}', dependencies: {[(c, c.lemma_) for c in test_token.children]}"

KeyError: 'nlp_docs'

## 2

## 3